In [2]:
import os
print(os.getcwd())
import sys
sys.path.append('C:\\Users\\theal\\PycharmProjects')
sys.path.append('C:\\Users\\theal\\PycharmProjects\\stockgym')

C:\Users\theal\PycharmProjects\stockgym\src\agents


In [7]:
import numpy as np
import gym
import random
from stockgym.src.envs.gym_up_and_to_the_right.up_and_to_the_right_env import UpAndToTheRightEnv

In [6]:
import numpy as np
import gym

class RandomActionAgent:
    def __init__(self, env):
        self.env = env

    def get_action(self):
        valid_actions = self.env.controller.get_valid_actions()
        action = np.random.choice(valid_actions)
        return action

    def run_episode(self):
        state = self.env.reset()
        done = False
        total_reward = 0

        while not done:
            action = self.get_action()
            next_state, reward, done, info = self.env.step(action)
            total_reward += reward
            state = next_state

        return total_reward

# Create the environment
env = UpAndToTheRightEnv(state_type="Basic", 
                         reward_type="FinalOnly",
                         num_prev_obvs=5,
                         price_movement_type="Linear",
                         offset_scaling=1.0,
                         scale=1.0,
                         slope=1.0,
                         noise=0.1,
                         starting_price=100,
                         num_steps=100,
                         multiple_units=True,
                         render=False)

# Create and run the agent
agent = RandomActionAgent(env)

for i in range(10):
    total_reward = agent.run_episode()
    print(f"Total reward from the episode {i + 1}: {total_reward}")

Total reward from the episode 1: 5.214808298583484
Total reward from the episode 2: 31.06877981127094
Total reward from the episode 3: -28.368293920511565
Total reward from the episode 4: 7.065593076331265
Total reward from the episode 5: -23.24411421274641
Total reward from the episode 6: 13.263251446985345
Total reward from the episode 7: 8.777043051346856
Total reward from the episode 8: 14.025798160414084
Total reward from the episode 9: -12.806673368474094
Total reward from the episode 10: 7.540730513039765


In [38]:
class QLearningAgent:
    def __init__(self, env, learning_rate=0.1, discount_factor=0.99, exploration_rate=1.0, exploration_decay=0.99):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.q_table = {}  # Initialize an empty Q-table

    def get_action(self, state):
        state_key = self.state_to_key(state)
        valid_actions = self.env.controller.get_valid_actions()
        self.initialize_state_in_q_table(state_key)

        if random.uniform(0, 1) < self.exploration_rate:
            return random.choice(valid_actions)  # Explore
        else:
            # Exploit by selecting the best valid action based on Q-values
            q_values = np.array([self.q_table[state_key][a] for a in range(self.env.action_space.n)])
            # Mask q_values of invalid actions with a very low number
            mask = np.ones(len(q_values)) * -np.inf
            mask[valid_actions] = 0
            q_values_masked = q_values + mask
            return np.argmax(q_values_masked)  # Exploit

    def update_q_table(self, state, action, reward, next_state):
        state_key = self.state_to_key(state)
        next_state_key = self.state_to_key(next_state)

        # Ensure both the current and next states are in the Q-table
        self.initialize_state_in_q_table(state_key)
        self.initialize_state_in_q_table(next_state_key)

        # Find the best next action from Q-table
        best_next_action = np.argmax(self.q_table[next_state_key])
        td_target = reward + self.discount_factor * self.q_table[next_state_key][best_next_action]
        td_error = td_target - self.q_table[state_key][action]
        self.q_table[state_key][action] += self.learning_rate * td_error


    def initialize_state_in_q_table(self, state_key):
        if state_key not in self.q_table:
            self.q_table[state_key] = [0 for _ in range(self.env.action_space.n)]


    def state_to_key(self, state):
        return tuple(state)


    def run_episode(self):
        state = self.env.reset()
        done = False
        total_reward = 0

        while not done:
            action = self.get_action(state)
            next_state, reward, done, info = self.env.step(action)
            self.update_q_table(state, action, reward, next_state)

            total_reward += reward
            state = next_state

        self.exploration_rate *= self.exploration_decay
        return total_reward

In [39]:
env = UpAndToTheRightEnv(state_type="Basic", 
                         reward_type="FinalOnly",
                         num_prev_obvs=5,
                         price_movement_type="Linear",
                         offset_scaling=1.0,
                         scale=1.0,
                         slope=1.0,
                         noise=0.1,
                         starting_price=100,
                         num_steps=100,
                         multiple_units=True,
                         render=False)

agent = QLearningAgent(env)

In [41]:
for i in range(10000):
    total_reward = agent.run_episode()
    print(f"Total reward from the episode {i + 1}: {total_reward}")

Total reward from the episode 1: 3628.906617370284
Total reward from the episode 2: 3664.7350314024898
Total reward from the episode 3: 3636.066938103739
Total reward from the episode 4: 3675.483326305874
Total reward from the episode 5: 3670.8611784806963
Total reward from the episode 6: 3672.0943798110643
Total reward from the episode 7: 3646.618105559278
Total reward from the episode 8: 3663.791595624963
Total reward from the episode 9: 3615.9814837606223
Total reward from the episode 10: 3652.4741069401107
Total reward from the episode 11: 3635.870927152221
Total reward from the episode 12: 3641.9696708570186
Total reward from the episode 13: 3634.627924306011
Total reward from the episode 14: 3651.6460072714467
Total reward from the episode 15: 3659.4890615297054
Total reward from the episode 16: 3619.2121001232917
Total reward from the episode 17: 3671.380555889068
Total reward from the episode 18: 3648.325808608175
Total reward from the episode 19: 3658.3899949874476
Total rewar

Total reward from the episode 157: 3666.383221767685
Total reward from the episode 158: 3709.1726434936318
Total reward from the episode 159: 3644.28783092696
Total reward from the episode 160: 3650.628247658105
Total reward from the episode 161: 3639.512747542283
Total reward from the episode 162: 3642.85109227845
Total reward from the episode 163: 3653.194806637851
Total reward from the episode 164: 3700.668575994901
Total reward from the episode 165: 3654.911553772048
Total reward from the episode 166: 3628.8977008378374
Total reward from the episode 167: 3623.586032588101
Total reward from the episode 168: 3653.0346046311615
Total reward from the episode 169: 3656.5888501394293
Total reward from the episode 170: 3655.258689053839
Total reward from the episode 171: 3600.6503947579536
Total reward from the episode 172: 3604.674992977434
Total reward from the episode 173: 3639.423771431012
Total reward from the episode 174: 3637.526251652429
Total reward from the episode 175: 3627.212

Total reward from the episode 320: 3654.1689975875647
Total reward from the episode 321: 3631.9814577116963
Total reward from the episode 322: 3630.7400099334463
Total reward from the episode 323: 3671.190174937059
Total reward from the episode 324: 3658.8686343375057
Total reward from the episode 325: 3684.0254349835327
Total reward from the episode 326: 3633.655786697888
Total reward from the episode 327: 3615.0818667867698
Total reward from the episode 328: 3642.0731685854885
Total reward from the episode 329: 3641.2773315976065
Total reward from the episode 330: 3670.4605849366194
Total reward from the episode 331: 3619.8405685937973
Total reward from the episode 332: 3627.577414139916
Total reward from the episode 333: 3633.7796346807136
Total reward from the episode 334: 3653.669209619149
Total reward from the episode 335: 3748.7892178633165
Total reward from the episode 336: 3651.7059123925374
Total reward from the episode 337: 3628.4737803798744
Total reward from the episode 33

Total reward from the episode 476: 3665.0347559736815
Total reward from the episode 477: 3682.5448264117504
Total reward from the episode 478: 3688.3152469259094
Total reward from the episode 479: 3642.0786601258537
Total reward from the episode 480: 3664.8152394292215
Total reward from the episode 481: 3672.846883698556
Total reward from the episode 482: 3590.0166408436567
Total reward from the episode 483: 3647.1014745207754
Total reward from the episode 484: 3664.809822169914
Total reward from the episode 485: 3673.887839436405
Total reward from the episode 486: 3673.6397326109204
Total reward from the episode 487: 3698.1549122759757
Total reward from the episode 488: 3617.8525873712138
Total reward from the episode 489: 3650.9014111698534
Total reward from the episode 490: 3681.2977229108305
Total reward from the episode 491: 3668.650095534545
Total reward from the episode 492: 3636.6673371215384
Total reward from the episode 493: 3634.0248490432054
Total reward from the episode 49

Total reward from the episode 645: 3632.5109083838256
Total reward from the episode 646: 3644.3291450932593
Total reward from the episode 647: 3643.9344325996076
Total reward from the episode 648: 3636.61122200942
Total reward from the episode 649: 3698.5670940152104
Total reward from the episode 650: 3615.626143323711
Total reward from the episode 651: 3679.4942419268173
Total reward from the episode 652: 3592.9809194424874
Total reward from the episode 653: 3639.926036148125
Total reward from the episode 654: 3668.3608043957515
Total reward from the episode 655: 3640.360076159908
Total reward from the episode 656: 3602.794709416223
Total reward from the episode 657: 3680.476328072002
Total reward from the episode 658: 3671.9688745367266
Total reward from the episode 659: 3663.411175967166
Total reward from the episode 660: 3657.5601825334215
Total reward from the episode 661: 3657.277601847904
Total reward from the episode 662: 3635.8101927337348
Total reward from the episode 663: 36

...

Total reward from the episode 9222: 3672.823418013457
Total reward from the episode 9223: 3632.7948338948313
Total reward from the episode 9224: 3627.4970932617034
Total reward from the episode 9225: 3695.245278261087
Total reward from the episode 9226: 3667.6982680767437
Total reward from the episode 9227: 3664.329681851778
Total reward from the episode 9228: 3692.44788145116
Total reward from the episode 9229: 3665.595058231953
Total reward from the episode 9230: 3667.0488504352916
Total reward from the episode 9231: 3644.4157147763467
Total reward from the episode 9232: 3658.6019323215396
Total reward from the episode 9233: 3637.087655287944
Total reward from the episode 9234: 3678.891630929903
Total reward from the episode 9235: 3691.231872110194
Total reward from the episode 9236: 3663.1121956575703
Total reward from the episode 9237: 3657.1880915467964
Total reward from the episode 9238: 3629.703880745936
Total reward from the episode 9239: 3669.2075898599956
Total reward from th

Total reward from the episode 9384: 3664.6341470408197
Total reward from the episode 9385: 3706.1242680744995
Total reward from the episode 9386: 3660.7962673939505
Total reward from the episode 9387: 3660.4787652967366
Total reward from the episode 9388: 3650.2339191058345
Total reward from the episode 9389: 3673.210629175435
Total reward from the episode 9390: 3633.7486969270317
Total reward from the episode 9391: 3670.3159680816834
Total reward from the episode 9392: 3641.0139472049086
Total reward from the episode 9393: 3620.3832404766636
Total reward from the episode 9394: 3664.9050373006685
Total reward from the episode 9395: 3665.8768524429597
Total reward from the episode 9396: 3648.1802975295795
Total reward from the episode 9397: 3657.9452488367206
Total reward from the episode 9398: 3626.228479016153
Total reward from the episode 9399: 3643.3084263963606
Total reward from the episode 9400: 3649.1397916842816
Total reward from the episode 9401: 3646.4142742659697
Total reward

Total reward from the episode 9538: 3655.6725708866666
Total reward from the episode 9539: 3685.5968840137175
Total reward from the episode 9540: 3656.292347434513
Total reward from the episode 9541: 3655.7526739274813
Total reward from the episode 9542: 3654.5158280443616
Total reward from the episode 9543: 3667.6540085303677
Total reward from the episode 9544: 3627.334434060183
Total reward from the episode 9545: 3652.380427104137
Total reward from the episode 9546: 3631.2511224787945
Total reward from the episode 9547: 3668.2522902400992
Total reward from the episode 9548: 3654.0054918458504
Total reward from the episode 9549: 3643.1794373974712
Total reward from the episode 9550: 3623.4538151390516
Total reward from the episode 9551: 3642.7058223854815
Total reward from the episode 9552: 3629.132618753614
Total reward from the episode 9553: 3665.660493511189
Total reward from the episode 9554: 3673.0083218480045
Total reward from the episode 9555: 3647.444465428518
Total reward fro

Total reward from the episode 9692: 3649.911980570828
Total reward from the episode 9693: 3690.2317095872995
Total reward from the episode 9694: 3661.2064447345338
Total reward from the episode 9695: 3684.316728511084
Total reward from the episode 9696: 3677.482068043887
Total reward from the episode 9697: 3680.5974869148986
Total reward from the episode 9698: 3614.586714324408
Total reward from the episode 9699: 3657.2962237603138
Total reward from the episode 9700: 3635.9200415343007
Total reward from the episode 9701: 3665.218115855916
Total reward from the episode 9702: 3654.5055262071514
Total reward from the episode 9703: 3681.3743994486454
Total reward from the episode 9704: 3660.504191248617
Total reward from the episode 9705: 3670.6748954729333
Total reward from the episode 9706: 3679.770070157879
Total reward from the episode 9707: 3676.6578527166876
Total reward from the episode 9708: 3623.0222421170133
Total reward from the episode 9709: 3639.523224645848
Total reward from 

Total reward from the episode 9846: 3631.010176349706
Total reward from the episode 9847: 3610.839779473827
Total reward from the episode 9848: 3651.509207371735
Total reward from the episode 9849: 3701.448486675133
Total reward from the episode 9850: 3656.303369624009
Total reward from the episode 9851: 3630.545091880952
Total reward from the episode 9852: 3611.4487218607387
Total reward from the episode 9853: 3705.596276654485
Total reward from the episode 9854: 3691.1438306139094
Total reward from the episode 9855: 3661.0100479534076
Total reward from the episode 9856: 3640.6945184960605
Total reward from the episode 9857: 3669.9599892866067
Total reward from the episode 9858: 3642.41502841101
Total reward from the episode 9859: 3646.1362024267696
Total reward from the episode 9860: 3655.8756581924613
Total reward from the episode 9861: 3629.546968096241
Total reward from the episode 9862: 3663.544257522909
Total reward from the episode 9863: 3622.7018186493706
Total reward from the